In [2]:
import warnings
warnings.filterwarnings('ignore', category = FutureWarning)
warnings.filterwarnings('ignore', category = DeprecationWarning)
warnings.filterwarnings('ignore', category = UserWarning)

In [3]:
import os
import re 
from ast import literal_eval
from random import seed
from itertools import chain
import multiprocessing
from nltk.tokenize import word_tokenize,WhitespaceTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import pandas as pd
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfTransformer,CountVectorizer
from xgboost import XGBClassifier
import string
import stop_words
from string import punctuation
from nltk.corpus import stopwords
import nltk.corpus.util
from sklearn.tree import DecisionTreeClassifier
from nltk.classify import NaiveBayesClassifier
from keras.layers import Dense,Dropout
from keras.models import Sequential

Using TensorFlow backend.


In [4]:
cores = multiprocessing.cpu_count() - 1

In [62]:
df = pd.read_csv(r'.\Desktop\Stream.csv',encoding = 'latin1')

In [90]:
df.columns

Index(['Sound Bite Text', 'Sentiment'], dtype='object')

In [706]:
#Length of Dataframe
len(df)

4525

In [11]:
#No. of rows containing text data
len(df[df["Sound Bite Text"].isna() == False])

15742

In [708]:
#Rows not containing any text data
len(df[df["Sound Bite Text"].isna() == True])

0

In [709]:
df[df["Sound Bite Text"].isna() == True]

,Post ID,Sound Bite Text,Ratings and Scores,Title,Source Type,Post Type,Media Type,URL,Domain,Published,...,Quoted Author Handle,Total Engagements,Post Comments,Post Likes,Post Shares,Post Views,Post Dislikes,Product Name,Product Hierarchy,Rating


In [710]:
#Rows not containing text data in "Sound Bite Text" column are dropped.
df.drop(df[df["Sound Bite Text"].isna() == True].index.values,axis = 0,inplace = True)

In [18]:
df = df.loc[df[df['Sentiment'].isna() == False].index.values]

In [91]:
#Creating seperate column for cleaned data(Without Hashtags)
df["soundBites"] = df["Sound Bite Text"].str.lower()

In [92]:
#Removing Hashtags
df["soundBites"] = df["soundBites"].apply(lambda row: re.sub(r'@[A-Za-z0-9_]+', '',row))

In [93]:
df['soundBites'] = df["soundBites"].apply(lambda row: re.sub('[^a-zA-Z0-9#-_]',' ',row))

In [94]:
df['soundBites'] = df['soundBites'].apply(lambda row: re.sub('#\w+',' ',row))
df['soundBites'] = df['soundBites'].apply(lambda row: re.sub('[â❤]',' ',row))

In [95]:
#Removing Non-words
df['soundBites'] = df["soundBites"].apply(lambda row: re.sub('[;+>\/<,.)¿(&?:?%*=-]+','',row))
#df["soundBites"] = df["soundBites"].apply(lambda row: re.sub('\W+'," ",row))

In [96]:
df["soundBites"] = df["soundBites"].apply(lambda row: re.sub('http\S+','',row))

In [97]:
#Removing Digits
df["soundBites"] = df["soundBites"].apply(lambda row: re.sub('\d+'," ",row))

In [98]:
df['soundBites'] = df["soundBites"].apply(lambda row: re.sub(r'\w+(.)$',' ',row))

In [99]:
df['soundBites'] = df["soundBites"].apply(lambda row: re.sub(r'\s+[.]',' ',row))

In [100]:
df['soundBites'] = df['soundBites'].apply(lambda row: row.split(' '))

In [101]:
df['soundBites']

0        [for, amarpatan, region, of, madhya, pradesh, ...
1        [consumer, called, up, and, asked, about, if, ...
2        [consumer, called, up, as, he, has, a, pack, o...
3        [retailer, called, in, again, and, said, that,...
4        [consumer, called, to, inquire, about, the, me...
5        [consumer, called, up, as, he, wanted, to, kno...
6        [consumer, called, us, and, shared, the, feedb...
7        [consumer, called, in, as, she, is, using, hig...
8        [source, nestlecom, , , , , firstname, sharad,...
9        [customer, is, looking, for, a, technician, fo...
10       [consumer, called, to, inquire, about, the, sh...
11       [source, nestlecom, , , , , firstname, jay, , ...
12       [consumer, is, looking, for, technician, for, ...
13       [hi, gudevening, , iam, nishan, karkera, here,...
14       [customer, called, up, as, he, want, distribut...
15       [consumer, is, looking, for, a, technician, as...
16       [consumer, called, saying, that, he, is, feedi.

In [102]:
#Filtering the empty strings from the list of tokens
df["soundBites"] = df["soundBites"].apply(lambda row: list(filter(None,row)))

In [103]:
#Creating list of stopwords
stopword = set(stopwords.words('english'))
stop_list = set(['maggi','maggie','also','nc','rz','njc','like','cz','gonna','carolina','garmin',
                 'screenshot','team','wish','provide','screen','rzd','moq','cgof','pcs','kis','ko','rt','customer','called',
                'informed','consumer'])
stopword.update(stop_list)

In [104]:
#Removing stopwords from tokens
df["soundBites"] = df["soundBites"].apply(lambda row :[item for item in row if item not in stopword])

In [105]:
df['soundBites']  = df['soundBites'].apply(' '.join)

In [106]:
wnl = WordNetLemmatizer()
df["soundBites"] = df["soundBites"].apply(lambda row: wnl.lemmatize(row))

In [63]:
df = data.copy()

In [64]:
#including Hashtags
df['soundBites_with_hashtags'] = df['Sound Bite Text'].str.lower()

In [73]:
df.drop([888,889,890],inplace = True)

In [71]:
df['soundBites_with_hashtags']

0      ic sent by :- @money_s1ngh  #mechanical #3rdye...
1      #foodie#coldcoffee#coffee#nescafe#colddrink#dr...
2                              ??nescafe, futala lake ..
3      #monsoondiaries#baarishkibaatein#pyaarbharamau...
4      49 ?250.00 prime tang lemon instant drink mix,...
5      all you need is just a perfect timing ??  rise...
6      @cafe_illusions_trimurti has a great collectio...
7      what else does one need ??? #netflix #netflixo...
8      hey fam ??  so today i have something for u ll...
9      ???? #simplybeingfit #strongasbhawna?? #nescaf...
10     #indianblogger #coffee #cafe #coffeetime #coff...
11     in the nestle-starbucks case, for example, exp...
12     yassssss we have a/c & tetley tea & nescafe & ...
13     by entering a marketing pact with starbucks, t...
14     it comes as nestle s nescafe brand of instant ...
15     nestle is taking a page from jab s strategy, a...
16     nestle also added niche brand chameleon cold-b...
17             location- nescaf

In [74]:
#Removing words not containing characters,hashtags or underscores
df['soundBites_with_hashtags'] = df["soundBites_with_hashtags"].apply(lambda row: re.sub('[^a-zA-Z0-9#-_]',' ',row))

In [75]:
df['soundBites_with_hashtags'] = df["soundBites_with_hashtags"].apply(lambda row: re.sub('@[a-zA-Z0-9_]+',' ',row))

In [76]:
df['soundBites_with_hashtags'] = df["soundBites_with_hashtags"].apply(lambda row: re.sub('[;+>.\/<,)❤☕₹⛩(&?:!%*=-]+','',row))

In [77]:
df['soundBites_with_hashtags'] = df["soundBites_with_hashtags"].apply(lambda row: re.sub('_{2,}','',row))

In [78]:
df["soundBites_with_hashtags"] = df["soundBites_with_hashtags"].apply(lambda row: re.sub('Â\x92',"\'",row))

In [79]:
df["soundBites_with_hashtags"] = df["soundBites_with_hashtags"].apply(lambda row: re.sub('Â\x91',"\'",row))

In [80]:
df["soundBites_with_hashtags"] = df["soundBites_with_hashtags"].apply(lambda row: re.sub('http\S','',row))

In [81]:
df['soundBites_with_hashtags']

0      ic sent by     #mechanical #3rdyear #branch #s...
1      #foodie#coldcoffee#coffee#nescafe#colddrink#dr...
2                                   nescafe futala lake 
3      #monsoondiaries#baarishkibaatein#pyaarbharamau...
4      49 25000 prime tang lemon instant drink mix 50...
5      all you need is just a perfect timing   rise a...
6        has a great collection of food and #nescafe ...
7      what else does one need  #netflix #netflixorig...
8      hey fam   so today i have something for u ll a...
9       #simplybeingfit #strongasbhawna #nescafe #lat...
10     #indianblogger #coffee #cafe #coffeetime #coff...
11     in the nestlestarbucks case for example expert...
12     yassssss we have ac  tetley tea  nescafe  a sw...
13     by entering a marketing pact with starbucks th...
14     it comes as nestle s nescafe brand of instant ...
15     nestle is taking a page from jab s strategy as...
16     nestle also added niche brand chameleon coldbr...
17                location nesc

In [82]:
df['soundBites_with_hashtags'] = df["soundBites_with_hashtags"].apply(lambda row: re.sub(r'[.]$',' ',row))

In [83]:
#removing dates and digits
df["soundBites_with_hashtags"] = df["soundBites_with_hashtags"].apply(lambda row: re.sub('\d+','',row))

In [84]:
df['soundBites_with_hashtags'] = df['soundBites_with_hashtags'].apply(lambda row: row.replace('-Minute','2-Minute'))

In [85]:
df['soundBites_with_hashtags'] = df['soundBites_with_hashtags'].apply(lambda row: row.replace('-minute','2-Minute'))

In [86]:
df['soundBites_with_hashtags'] = df['soundBites_with_hashtags'].apply(lambda row: re.split(r'(#\w+)',row))

In [87]:
df['soundBites_with_hashtags'] = df['soundBites_with_hashtags'].apply(' '.join)

In [88]:
wnl = WordNetLemmatizer()
df["soundBites_with_hashtags"] = df["soundBites_with_hashtags"].apply(lambda row: wnl.lemmatize(row))

In [89]:
df['soundBites_with_hashtags'] = df['soundBites_with_hashtags'].apply(lambda row: re.split(r' ',row))

In [90]:
#Filtering the empty strings from the list of tokens
df["soundBites_with_hashtags"] = df["soundBites_with_hashtags"].apply(lambda row: list(filter(None,row)))

In [758]:
stopword = set(stopwords.words('english'))
stopword_update = [item for item in stop_words.get_stop_words('en') if item not in nltk.corpus.stopwords.words('english')]
stop_list = set(['maggi','maggie','#maggi','#maggie','also','njc','nc','rz','use','like','carolina','food','love','garmin','new',
                 'raha','pm','tags','pak','across','ki','hi','screenshot','team','wish','provide','screen',
                 'pictwittercomclsauwtd',
                 '#foodie','#instafood','#foodblogger','tag','rt','updates','reuters','#fooddiaries','#foodisbae','#foodography',
                '#likeforlike','#followforfollow','follow','join','rs','#foodlove','delicious','#foodgasm','#foodielife','simpsons'
                 '#foodblogger','#merimaggie','#merimaggi','try','#indianblogger','#maggilove','#njcs','#maggielove','#foodporn',
                '#food','#foodie','#foodporn','#zomato','#maniamaggi','#tastymaggi','#maniamaggie','#tastymaggie','hai','u','km',
                 'tasty','yummy','#yummylicious','#yummilicious','#foddy','#foodgram','#maggilover','#maggielover','#tasty','rda',
                 '#yummy','#foddie','#swiggyindia','#zomatoindia','#swiggy','#foodstagram','#instagood','#indianfood','#likelike',
                 '#love','#delicious','#foodlover,''#picoftheday'',#noodles','#eeeats','#eeeeeats','#foodpics','#picoftheday',
                'noodles','#sodelhi','#likelike','#nomnom','#buzzfoodfeed','#fgram','#eatingforinsta','#saadidilli','kuch',
                '#saddidilli','#ll','#flt','#foodphotography','#foodtalkindia','#delhifoodie','#foodlover','#maggilovers',
                '#maggielovers','#foodtalkindia','#delhifoodblogger','#delhigram','#instagram','#instagood','#photooftheday',
                '#instadaily','#followfollow','#fgrams','#streetfood','#igers','#','#iger','#followme','#foodies','#likes',
                 '#instalike','#instalike','#indianfoodbloggers','indianfoodblogger','#foodshot','#foodcoma','gunsmoke','#noodles',
                '#vscodelhi','#foodblog','#foodiegram','#foodphoto','#delhi_igers','#delhifood','#delhidiaries','#mumbaifood',
                 '#mumbaidiaries','#foodoftheday','#indianfoodie','#foodaholic','#foods','#vscocam','#vscofood','#likeforfollow',
                '#delhifoodie','#streetfoods','#eat','#yum','#follow','th','us','#follow','#india','#ahmedabad','#chennai','#delhi',
                '#mumbai','#maggiloverforever','#vsco','#likesforlikes','#buzzfeedfood','#delhiblogger','#dfordelhi',
                '#tastyfood','#foodiesofinstagram','#knowyourmaggi'])
stopword.update(stop_list),
stopword.update(set(stopword_update))
stopword.update(set(string.ascii_lowercase))

In [759]:
#Removing stopwords from tokens
df["soundBites_with_hashtags"] = df["soundBites_with_hashtags"].apply(lambda row: [item for item in row if item not in stopword])

In [91]:
df["soundBites_with_hashtags"] = df["soundBites_with_hashtags"].apply(' '.join)

In [93]:
df["soundBites_with_hashtags"]

0      ic sent by #mechanical #rdyear #branch #simple...
1      #foodie #coldcoffee #coffee #nescafe #colddrin...
2                                    nescafe futala lake
3      #monsoondiaries #baarishkibaatein #pyaarbharam...
4      prime tang lemon instant drink mix g pouch pri...
5      all you need is just a perfect timing rise and...
6      has a great collection of food and #nescafe s ...
7      what else does one need #netflix #netflixorigi...
8      hey fam so today i have something for u ll all...
9      #simplybeingfit #strongasbhawna #nescafe #latt...
10     #indianblogger #coffee #cafe #coffeetime #coff...
11     in the nestlestarbucks case for example expert...
12     yassssss we have ac tetley tea nescafe a swimm...
13     by entering a marketing pact with starbucks th...
14     it comes as nestle s nescafe brand of instant ...
15     nestle is taking a page from jab s strategy as...
16     nestle also added niche brand chameleon coldbr...
17                 location nes

In [762]:
df.to_csv('May_Personas_Clean.csv')

In [ ]:
df['soundBites_with_hashtags'][9829]

In [ ]:
data = pd.read_csv("Maggi_Clean2.csv")

In [91]:
data = df[df['Sentiments'].isna() == False]

In [94]:
data.loc[data['Sentiments'] == 'None','Sentiments'] = 'Neutral'

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [95]:
df.to_csv('.\Desktop\Stream.csv')

In [98]:
#Creating seperate dataset for Classification
dataset = pd.DataFrame(data['soundBites'])
dataset['Sentiments'] = data['Sentiments']

In [ ]:
#https://docs.python.org/2/library/ast.html#ast.literal_eval
dataset['soundBites'] = dataset['soundBites'].apply(lambda row: literal_eval(''.join(row)))

In [ ]:
#Creating a string from list of tokens
dataset['soundBites'] = dataset['soundBites'].apply(' '.join)

In [44]:
dataset = df.copy()
dataset.rename(columns = {'Sentiment': 'Sentiments'},inplace = True)

In [108]:
#Splitting training and testing data
x_train,x_test,y_train,y_test = train_test_split(dataset['soundBites'],dataset['Sentiments'], random_state = 20, test_size = 0.3)

In [109]:
#Applying CountVectorizer to get token count
cv = CountVectorizer()
X_train = cv.fit_transform(x_train)
X_test = cv.transform(x_test)

In [110]:
len(x_train) + len(x_test)

4630

In [111]:
#Applying TFIDF
tfidf = TfidfTransformer()
X_train = tfidf.fit_transform(X_train)
X_test = tfidf.transform(X_test)

In [112]:
#Converting sparse training and testing datasets to dense matrices for classification
X_train = X_train.todense()
X_test = X_test.todense()
#Classification using Multinomial Naive Bayes
classifier = MultinomialNB()
classifier.fit(X_train,y_train)
prediction = classifier.predict(X_test)
accuracy_score(y_true = y_test, y_pred = prediction)

0.8898488120950324

In [75]:
#Classifcation using Gaussian Naive Bayes
classifier = GaussianNB()
classifier.fit(X_train,y_train)
prediction = classifier.predict(X_test)
accuracy_score(y_true = y_test, y_pred = prediction)

0.5673146148308136

In [113]:
#Classification Using Decision Tree Classifier
clfdt = DecisionTreeClassifier(random_state = 42)
clfdt.fit(X_train,y_train)
prediction = clfdt.predict(X_test)
accuracy_score(y_true = y_test,y_pred = prediction)
print("Accuracy: ",accuracy_score(y_pred = prediction,y_true = y_test))
confusion_matrix(y_true = y_test, y_pred = prediction)

Accuracy:  0.9006479481641468


array([[ 17,  95,   5],
       [  9, 871,  19],
       [  1,   9, 363]], dtype=int64)

In [77]:
#Classification Using Support Vector Classifier
clfSVM = LinearSVC()
clfSVM.fit(X_train,y_train)
prediction = clfSVM.predict(X_test)
print("Accuracy: ",accuracy_score(y_pred = prediction,y_true = y_test))
confusion_matrix(y_true = y_test, y_pred = prediction)

Accuracy:  0.912167026637869


array([[ 16,  97,   4],
       [  0, 890,   9],
       [  0,  12, 361]], dtype=int64)

In [78]:
#Classification Using Random Forest Classifier
rfclf = RandomForestClassifier(random_state = 0)
rfclf.fit(X_train,y_train)
prediction = rfclf.predict(X_test)
accuracy_score(y_true = y_test, y_pred = prediction)
print("Accuracy: ",accuracy_score(y_pred = prediction,y_true = y_test))
confusion_matrix(y_true = y_test, y_pred = prediction)

Accuracy:  0.8984881209503239


array([[  9,  98,  10],
       [  2, 880,  17],
       [  0,  14, 359]], dtype=int64)

In [ ]:
len(dataset['Sentiments']=='P')

In [60]:
#Classification using XGBoost Classifier
xgb = XGBClassifier(random_state = 21)
xgb.fit(X_train,y_train)
prediction = xgb.predict(X_test)
accuracy_score(y_true = y_test, y_pred = prediction)
print("Accuracy: ",accuracy_score(y_pred = prediction,y_true = y_test))
confusion_matrix(y_true = y_test, y_pred = prediction)

Accuracy:  0.9136069114470843


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


array([[ 20,  96,   1],
       [  0, 896,   3],
       [  4,  16, 353]], dtype=int64)

In [ ]:
X = pd.concat([pd.DataFrame(X_train),pd.DataFrame(X_test)])
Y = pd.concat([y_train,y_test])

In [108]:
clfdt = DecisionTreeClassifier(random_state = 42)
clfdt.fit(X_test,y_test)
prediction = clfdt.predict(X_train)
accuracy_score(y_true = y_train,y_pred = prediction)
print("Accuracy: ",accuracy_score(y_pred = prediction,y_true = y_train))
confusion_matrix(y_true = y_train, y_pred = prediction)

Accuracy:  0.6996830427892234


array([[  0,   1,   8],
       [  1, 123, 263],
       [  3, 103, 760]], dtype=int64)

In [76]:
Y_train_.shape

(1262, 4)

In [61]:
labelEncoder = LabelEncoder()
Y_train = labelEncoder.fit_transform(y_train)
Y_test = labelEncoder.transform(y_test)

In [86]:
labelEncoder.classes_

array(['N', 'Neutral', 'P', 'P0'], dtype=object)

In [62]:
one_hot_encoder = OneHotEncoder()
Y_train_ = one_hot_encoder.fit_transform(Y_train.reshape(-1,1))
Y_test_ = one_hot_encoder.transform(Y_test.reshape(-1,1))

In [64]:
# Using Deeplearning for Classification
seed(0)
model = Sequential()
model.add(Dense(output_dim =  32, input_shape = (X_train.shape[1],),activation = 'relu'))
model.add(Dense(output_dim = 32, activation = 'relu'))
model.add(Dense(output_dim = 32, activation = 'relu'))
model.add(Dense(output_dim = 32, activation = 'relu'))
model.add(Dense(output_dim = 4, activation = 'softmax'))
model.compile(optimizer = 'adagrad', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.fit(X_train, Y_train_, batch_size = 100, epochs = 100)
y_pred = model.predict(X_test)

Epoch 1/100
3241/3241 [==============================] - 2s 563us/step - loss: 0.6463 - acc: 0.7877
Epoch 2/100
3241/3241 [==============================] - 0s 125us/step - loss: 0.2941 - acc: 0.9003
Epoch 3/100
3241/3241 [==============================] - 0s 130us/step - loss: 0.2481 - acc: 0.9087
Epoch 4/100
3241/3241 [==============================] - 0s 125us/step - loss: 0.2226 - acc: 0.9272
Epoch 5/100
3241/3241 [==============================] - 0s 130us/step - loss: 0.2074 - acc: 0.9324
Epoch 6/100
3241/3241 [==============================] - 0s 130us/step - loss: 0.1975 - acc: 0.9352
Epoch 7/100
3241/3241 [==============================] - 0s 125us/step - loss: 0.1922 - acc: 0.9358
Epoch 8/100
3241/3241 [==============================] - 0s 130us/step - loss: 0.1888 - acc: 0.9371
Epoch 9/100
3241/3241 [==============================] - 0s 125us/step - loss: 0.1874 - acc: 0.9367
Epoch 10/100
3241/3241 [==============================] - 0s 130us/step - loss: 0.1877 - acc: 0.9371

In [65]:
print('Accuracy = ',accuracy_score(y_true = Y_test, y_pred = y_pred.argmax(axis = 1)))
print(confusion_matrix(y_true = Y_test, y_pred = y_pred.argmax(axis = 1)))

Accuracy =  0.9092872570194385
[[ 15  96   6]
 [  3 886  10]
 [  2   9 362]]


In [65]:
analyzer = SentimentIntensityAnalyzer()

In [ ]:
data = pd.read_csv("Maggi_Clean.csv")

In [ ]:
for item in data['Sound Bite Text']:   
    i = data.index.values[data['Sound Bite Text'] == item]
    scores = analyzer.polarity_scores(item)
    if (scores['compound'] < 0.05) & (scores['compound'] > -0.05):
        data.loc[i,'vader_Senti'] = 'None'
    elif scores['compound'] < -0.05:
        data.loc[i,'vader_Senti'] = 'N'
    elif scores['compound'] > 0.05:
        data.loc[i,'vader_Senti'] = 'P'

In [ ]:
data[data['Sentiment'].isna()]

In [ ]:
dataset.shape

In [ ]:
le = LabelEncoder()
vader_in = le.fit_transform(data['Sentiment'])
vader_out = le.transform(data['vader_Senti'])

In [ ]:
data['vader_Senti'].unique()

In [ ]:
accuracy_score(y_true = vader_out, y_pred  = vader_in)

In [ ]:
confusion_matrix(y_true = vader_out,y_pred = vader_in)

In [ ]:
data.loc[data['vader_Senti'] == 'N','Sound Bite Text'][42]

In [ ]:
!pip install --upgrade setuptools

In [143]:
train = df.loc[df['Sentiment'].dropna(axis = 0).index.values]

In [147]:
X_train = train['soundBites']
Y_train = train['Sentiment']

In [151]:
test = df.loc[df['Sentiment'].isna() == True]
X_test = df['soundBites']

In [152]:
cv = CountVectorizer()
X_train = cv.fit_transform(X_train)
X_test = cv.transform(X_test)

In [154]:
tfidf = TfidfTransformer()
X_train = tfidf.fit_transform(X_train)
X_test = tfidf.transform(X_test)

In [156]:
X_train = X_train.todense()
X_test = X_test.todense()

In [159]:
#DecisionTreeClassifier
clf = DecisionTreeClassifier()
clf.fit(X_train,Y_train)
df['Decision Tree'] = clf.predict(X_test)

In [ ]:
#Classification using XGBoost Classifier
xgb = XGBClassifier(random_state = 21,n_jobs = cores)
xgb.fit(X_train,Y_train)
df['XGB'] = xgb.predict(X_test)

In [162]:
#Classification Using Support Vector Classifier
clfSVM = LinearSVC()
clfSVM.fit(X_train,Y_train)
df['SVM'] = clfSVM.predict(X_test)

In [176]:
df

,Sound Bite Text,Sentiment,soundBites,Decision Tree,SVM
0,"For AMARPATAN region of Madhya Pradesh, we wis...",P,amarpatan region madhya pradesh nestle product...,P,P
1,Consumer called up and asked about if he can g...,Neutral,asked give nestogen months baby please consult,Neutral,Neutral
2,Consumer called up as he has a pack of NESTLÃ...,Neutral,pack nestl cerelac wheat apple cherry wanted k...,Neutral,Neutral
3,Retailer called in again and said that his con...,N,retailer said concern registered previously go...,N,N
4,Consumer called to inquire about the method of...,Neutral,inquire method preparation nan excella pro mon...,Neutral,Neutral
5,Consumer called up as he wanted to know about ...,Neutral,wanted know expiration date minute noodles gms...,Neutral,Neutral
6,Consumer called us and shared the feedback for...,P,us shared feedback nestle products happy nestle,P,P
7,Consumer called in as she is using HIGH PROTEI...,N,using high protein resource however available ...,N,N
8,SOURCE: nestle.com \r\n\r\nFIRSTNAME: Sharad\r...,P,source nestlecom firstname sharad lastname tan...,P,P
9,Customer is looking for a technician for Two v...,N,looking technician two vending machine showing...,N,N


In [167]:
accuracy_score(y_true = df['SVM'],y_pred = df['Decision Tree'])

0.7612120442129335